# Training data preparation
In this project, we have human subject data sampled annually. The human subjects are identified uniquelly via the variable `hhidpn` and the year via `wave`. 
* Features: 
    * categorical: `race`, 
    * semi-categorical/numerical: `educ`, `cohort`, `female` (binary)
    * continous/numerical: `age`, `activity`, `freqpray`, `mwi`, `assets`, `income`, `social` (?), `physd` (?), `formal`, `informal`, `depress`, `memoryp` (binary), `everyday`, `majordisc`, `cumulative`,  
    * not in use: `wave` (year), `dead` (binary), 
* Targets: `recall`, `seven`, `bwcount`. they are added into `cogtot`. for most people, they all drop along time as our brain degrades. 

Some variables are not time-variant: `female`, `race`, `educ`, `cohort`. 


In [1]:
numerical_features = ["female", "age", "educ", "cohort", "activity", 
    "freqpray", "mwi", "assets", "income", "social", "physd", "formal", 
    "informal", "depress", "memoryp", "everyday", "majordisc", "cumulative"]
# try to exclude: everyday, majordisc, cumulative 
categorical_features = ["race"]
feature_columns = numerical_features + categorical_features
target_columns = ["recall", "seven", "bwcount"]
identifier_columns = ["hhidpn", "wave", "dead"]
keep_columns = feature_columns + target_columns +  identifier_columns

# What we want
We want to predict what factors significantly impact the cognitive functioning over time. How do you quantify the significance...It can be measured on the change of a dependent variable. 

# Data filtering 
* Only rows with `dead==0` and `age>0` are used. This reduces total number of rows from 50k to 20k. 
* Drop any row where targets are strings or NaN. 
* Drop any subject's data is the number of wave is below 2. 

# ideas for the future
1. Transfer/multi-task learning w.r.t. to group identifiers, e.g., race. 



## Loading the data

In [1]:
import pandas
import numpy
import tqdm
import pickle 

In [2]:
df = pandas.read_csv("ML_social.csv")

/shared-libs/python3.9/py-core/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3139: DtypeWarning: Columns (5,24,28,29,30,31,32,33,34,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.columns

Index(['hhidpn', 'wave', 'race', 'female', 'educ', 'cohort', 'recall', 'seven',
       'bwcount', 'age', 'assets', 'income', 'activity', 'formal', 'informal',
       'hbp', 'diab', 'lung', 'angina', 'myocard', 'heartfail', 'stroke',
       'arthritis', 'cogtot', 'mar', 'depress', 'newwork', 'dead', 'lb036a',
       'lb036b', 'lb036c', 'lb036d', 'lb036e', 'lb036f', 'lb036g', 'freqpray',
       'attribution', 'lb030a', 'lb030b', 'lb030c', 'lb030d', 'lb030e',
       'lb030f', 'lb030g', 'lb021b', 'lb021d', 'lb021f', 'lb021h', 'd1', 'd2',
       'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'ltact_i', 'mddact_i', 'vigact_i',
       'mwi', 'obese', 'memoryp', 'adrd', 'physd', 'social', 'everyday',
       'disca', 'discb', 'discc', 'discd', 'disce', 'discf', 'majordisc',
       'cumulative'],
      dtype='object')

In [4]:
df['age'].mean()

67.20632366520015

In [4]:
# print (target_columns)
# for user in df['hhidpn'].unique():
#     user_df = df[df['hhidpn']==user]
#     if user_df[target_columns].isna().any(axis=1).any():
#         print (user_df)
#         print (user_df[target_columns].isna().any(axis=1).index)
#         print (user_df[target_columns].isna())
#         print (user_df[target_columns].isna().any(axis=1))
#         na_judge = user_df[target_columns].isna().any(axis=1)
#         print (na_judge[na_judge].index)
#         print (user_df.dropna(subset=target_columns))
#         break 

## Filtering columns 
We will only use columns below from the CSV file. 

In [5]:
df = df[keep_columns]
print ("Columns kept:", df.columns)

Columns kept: Index(['female', 'age', 'educ', 'cohort', 'activity', 'freqpray', 'mwi',
       'assets', 'income', 'social', 'physd', 'formal', 'informal', 'depress',
       'memoryp', 'everyday', 'majordisc', 'cumulative', 'race', 'recall',
       'seven', 'bwcount', 'hhidpn', 'wave', 'dead'],
      dtype='object')


## Dropping rows, round 1: based on target Nan and `dead` and `age`

In [6]:
print (f"Original number of rows: {len(df)}, and subjects: {len(df['hhidpn'].unique())}") 
df=df.dropna(subset=target_columns)
print (f"After dropping rows that has NaN in target columns, \
        \n\t in remaining data, \
        number of rows:  {len(df)}, and subjects: {len(df['hhidpn'].unique())} ")
df = df[df['dead']==0]
print (f"After dropping rows where dead column is 0,  \n\t in remaining data, \
        number of rows:  {len(df)}, and subjects: {len(df['hhidpn'].unique())} ")
df = df[df['age']>0]
print (f"After dropping rows where age column is not a number: \n\t in remaining data, \
        number of rows:  {len(df)}, and subjects: {len(df['hhidpn'].unique())} ")


Original number of rows: 407260, and subjects: 50971
After dropping rows that has NaN in target columns,         
	 in remaining data,         number of rows:  138658, and subjects: 30116 
After dropping rows where dead column is 0,  
	 in remaining data,         number of rows:  127029, and subjects: 29965 
After dropping rows where age column is not a number: 
	 in remaining data,         number of rows:  127029, and subjects: 29965 


## Remove `.*` strings and numericalize all data
Replace `.s`, `.r`, `.d` and `.m` to `numpy.nan`. 

#### Test `.*` strings

In [7]:
# .s and .m examples 
df['depress'].iloc[3770:3780]

8676    0
8677    0
8679    0
8680    0
8681    0
8682    0
8683    0
8687    0
8688    0
8695    0
Name: depress, dtype: object

In [8]:
# .r example 
df['memoryp'].iloc[13130:13140]

30322    NaN
30323    NaN
30324    NaN
30325    NaN
30326    NaN
30328      0
30329      0
30330    NaN
30331    NaN
30332    NaN
Name: memoryp, dtype: object

In [9]:
# .x example 
df['recall'].iloc[5100:5110]

11492    16
11504     6
11505    11
11506     6
11513    10
11514     6
11515    10
11516    10
11550     7
11551     6
Name: recall, dtype: object

In [10]:
# .d example 
df['memoryp'].iloc[31290:31300]

76624    NaN
76625    NaN
76626    NaN
76628      0
76629      0
76630    NaN
76631    NaN
76632    NaN
76633    NaN
76634    NaN
Name: memoryp, dtype: object

#### Actually removing them and turning all columns to numeric 

In [11]:
for special_string in [".s", ".m", ".r", ".d", ".x"]:
    df = df.replace(special_string, numpy.nan)

for column in df.columns: 
    # print (type(df_test[column].dtype))
    if df[column].dtype == numpy.object_:
        # print (column)
        df[column] = pandas.to_numeric(df[column])

# verify it works globally
# print (df_test.dtypes)

# verify that it really works locally 
# print ("before", df4['memoryp'].iloc[31295:31300])
# print ("After", df_test['memoryp'].iloc[31295:31300])


# Test code. Ensure no .s, .m, .r, .d any more. 
# for index, row in df.iterrows():
#     for i in range(len(row)):
#         if row.iloc[i]  == ".s":
#             print (row)
#             break

## Dropping rows, round 2: `NaN` row introduced after replacing `.*` strings

In [12]:
print ("number of rows, before: ", len(df))
print ("number of subjects, before: ", len(df['hhidpn'].unique()))
df=df.dropna(subset=target_columns)
print ("number of rows, after: ", len(df))
print ("number of subjects, after: ", len(df['hhidpn'].unique()))


number of rows, before:  127029
number of subjects, before:  29965
number of rows, after:  123025
number of subjects, after:  29648


## Dropping rows, round 3: remove subjects with less than 2 waves. 

In [13]:
print ("Original number of subjects:", len(df['hhidpn'].unique()), "Original number of rows:", len(df) )

okay_row, okay_subject, seq_length = 0, 0,  []  

qualified_subject = []
for subjectID in tqdm.tqdm(df['hhidpn'].unique()):
    subject_df = df[df['hhidpn']==subjectID]
    if len(subject_df) > 1:
        okay_subject += 1
        okay_row += len(subject_df) 
        seq_length.append(len(subject_df) )
        qualified_subject.append(int(subjectID)) 

        # debug 
        # if okay_subject> 10 :
        #     break 

df = df[df['hhidpn'].isin(qualified_subject)] 

print ("\nNumber of okayed subjets:", okay_subject, "Number of okayed rows:", okay_row, "average length (years) of a subject:",  sum(seq_length)/len(seq_length))


Original number of subjects: 29648 Original number of rows: 123025
100%|██████████| 29648/29648 [00:18<00:00, 1635.34it/s]
Number of okayed subjets: 26178 Number of okayed rows: 119555 average length (years) of a subject: 4.567002826801131



In [21]:
df.dtypes

female        float64
age           float64
educ          float64
cohort        float64
activity      float64
freqpray      float64
mwi           float64
assets        float64
income        float64
social        float64
physd         float64
formal        float64
informal      float64
depress       float64
memoryp       float64
everyday      float64
majordisc     float64
cumulative    float64
race          float64
recall        float64
seven         float64
bwcount       float64
hhidpn          int64
wave            int64
dead          float64
dtype: object

## Filling NA

In [14]:
# fill N/A
df = df.fillna(0.5)

# Creating the X,Y pairs 

X or Y is a 1-D list of 2D numpy.ndarry, looks like this: 
```python
[
    [# subject 1 
        [feature_1, feature_2, feature_3], # year 1 
        [], # year 2
        ...
        [], # year M  
    ], 
    [
        [], # year 1 
        [], # year 2
        ...
        [], # year M  
    ], # subject 2
    ...
    [
        [], # year 1 
        [], # year 2
        ...
        [], # year M  
    ], # subject N  
]
```


In [15]:
len(numerical_features)

18

In [16]:
import sklearn, sklearn.preprocessing 

def pack_into_time_series(df, numerical_features, categorical_features, target_columns):
    X, Y = [], [] # X and y are the training inputs and targets. 
    # X, Y = numpy.array([]), numpy.array([]) # X and y are the training inputs and targets. 
    # X or Y is a 1D list of 2D numpy arrays. X[subject] -> array[year][feature]

    # one hot encoding for categorical features
    one_hot_encoder = sklearn.preprocessing.OneHotEncoder(sparse=False)
    categorical_transformed = one_hot_encoder.fit_transform(df[categorical_features].to_numpy()) 
    one_hot_encoded_feature_names = one_hot_encoder.get_feature_names_out(categorical_features)
    ohe_df = pandas.DataFrame(categorical_transformed, 
                                columns=one_hot_encoded_feature_names, 
                                index = df.index) # without the index, concat misalin unless using ignore_index
    feature_columns = numerical_features + one_hot_encoded_feature_names.tolist()
    df = df.drop(categorical_features, axis=1)
    df = pandas.concat([df, ohe_df], axis=1)
      
    # # Columns of type Object to float64 
    # for a_numerical_feature in numerical_features: 
    #     if df[a_numerical_feature].dtype not in ['int', 'float']:
    #         df[a_numerical_feature] = df[a_numerical_feature].astype(float)

    # min-max scale for numerical features 
    feature_scaler = sklearn.preprocessing.MinMaxScaler()
    df[numerical_features] = feature_scaler.fit_transform(df[numerical_features])

    target_scaler = sklearn.preprocessing.MinMaxScaler()
    df[target_columns] = feature_scaler.fit_transform(df[target_columns])

    for subjectID in tqdm.tqdm(df['hhidpn'].unique()):
        subject_df = df[df['hhidpn'] == subjectID]
        subject_X, subject_Y = subject_df[feature_columns].to_numpy(), subject_df[target_columns].to_numpy()
        X.append(subject_X)
        Y.append(subject_Y)
        # numpy.append(X, subject_X, axis=0)
        # numpy.append(Y, subject_Y, axis=0)
        # break 

    X, Y  = numpy.array(X), numpy.array(Y)

    return X, Y

X, Y  = pack_into_time_series(df, numerical_features, categorical_features, target_columns)
print (X.shape, Y.shape)

pickle.dump([X, Y], open("XY.pickle", 'wb'))

100%|██████████| 26178/26178 [00:50<00:00, 521.40it/s]
/tmp/ipykernel_74/1321518648.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X, Y  = numpy.array(X), numpy.array(Y)
(26178,) (26178,)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=38ce621b-4696-4047-8b25-0501b493ce55' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>